# Ultrack I2K 2023 - Multiple hypotheses tracking

## Setup Dependencies

This step is only necessary if you are on Colab or don't have the required packages.
When using Colab, the runtime must be initialized.

Install the necessary packages for cell tracking, image processing, and visualization.
Uncomment and run the following commands to install all required packages.

In [1]:
# !pip install stackview cellpose 'napari[all]' ultrack ipycanvas==0.11 cucim
# !pip install git+https://github.com/Janelia-Trackathon-2023/traccuracy

## Download Dataset

Download the Fluo-C2DL-Huh7 dataset from the [Cell Tracking Challenge](celltrackingchallenge.net), which contains fluorescence microscopy images for cell tracking.

The dataset will be used for demonstrating the segmentation and tracking workflow.

!wget -nc http://data.celltrackingchallenge.net/training-datasets/Fluo-C2DL-Huh7.zip
!unzip -n Fluo-C2DL-Huh7.zip

## Import Libraries

Import the libraries needed for reading images, processing them, cell segmentation, tracking, and performance metrics. 

In [2]:
from pathlib import Path
from typing import Dict

import pandas as pd
import numpy as np
import stackview
from dask.array.image import imread
from numpy.typing import ArrayLike
from rich import print

from traccuracy import run_metrics
from traccuracy.loaders import load_ctc_data
from traccuracy.matchers import CTCMatched
from traccuracy.metrics import CTCMetrics

from ultrack import track, to_tracks_layer, tracks_to_zarr, to_ctc
from ultrack.utils import labels_to_edges
from ultrack.config import MainConfig
from ultrack.imgproc import normalize
from ultrack.imgproc.segmentation import Cellpose
from ultrack.utils.array import array_apply

## Colab or Local

Check if the tutorial is running in a Google Colab environment or on a local machine. Depending on the environment, initialize a viewer for visualizations.

In [3]:
COLAB = True
# COLAB = False

if COLAB:
    viewer = None

    # fixes colab encoding error
    import locale
    locale.getpreferredencoding = lambda: "UTF-8"

    # enabling colab output
    try:
        from google.colab import output
        output.enable_custom_widget_manager()
    except ModuleNotFoundError as e:
        print(e)
else:
    import napari
    viewer = napari.Viewer()

No module named 'google'

## Load Data

Load the Fluo-C2DL-Huh7 dataset into memory. This dataset contains TIFF images which we will visualize using stackview if running on Colab or napari if running locally.

In [4]:
dataset = "02"
path = Path("Fluo-C2DL-Huh7") / dataset
image = imread(str(path / "*.tif"))

if COLAB:
    display(stackview.slice(image))
else:
    viewer.add_image(image)

## Configuration

We'll use the same configuration as in the previous example, except for `config.segmentation_config.min_frontier`, which we decreased.

The `min_frontier` merges regions with an average contour lower than the provided value.
Since the contours are combined by averaging, the previous value of 0.1 removed relevant segments from the candidate hypotheses.

As a reminder, its documentation can be found [here](https://github.com/royerlab/ultrack/blob/main/ultrack/config/README.md).

In [5]:
config = MainConfig()

# Candidate segmentation parameters
config.segmentation_config.n_workers = 8
config.segmentation_config.min_area = 2500
config.segmentation_config.min_frontier = 0.05  # NOTE: this parameter is not the same as in intro.ipynb

# Setting the maximum number of candidate neighbors and maximum spatial distance between cells
config.linking_config.max_neighbors = 5
config.linking_config.max_distance = 100
config.linking_config.n_workers = 8

# Adding absurd weight to division because there's no diving cell
config.tracking_config.division_weight = -100
# Very few tracks enter/leave the field of view, increasing penalization
config.tracking_config.disappear_weight = -1
config.tracking_config.appear_weight = -1

print(config)

MainConfig(
    data_config=DataConfig(working_dir=PosixPath('.'), database='sqlite', address=None, n_workers=1),
    segmentation_config=SegmentationConfig(
        threshold=0.5,
        min_area=2500,
        max_area=1000000,
        min_frontier=0.05,
        anisotropy_penalization=0.0,
        max_noise=0.0,
        ws_hierarchy=<function watershed_hierarchy_by_area at 0x1675e7f60>,
        n_workers=8
    ),
    linking_config=LinkingConfig(
        n_workers=8,
        max_neighbors=5,
        max_distance=100,
        distance_weight=0.0,
        z_score_threshold=5.0
    ),
    tracking_config=TrackingConfig(
        appear_weight=-1,
        disappear_weight=-1,
        division_weight=-100,
        dismiss_weight_guess=None,
        include_weight_guess=None,
        window_size=None,
        overlap_size=1,
        solution_gap=0.001,
        time_limit=36000,
        method=0,
        n_threads=-1,
        link_function='power',
        power=4,
        bias=-0.0
    )
)

## Cellpose Segmentation

The same function as `intro.ipynb` to segment cells within each frame.

In [6]:
cellpose = Cellpose(model_type="cyto2", gpu=True)

def predict(frame: ArrayLike, gamma: float) -> ArrayLike:
    norm_frame = normalize(np.asarray(frame), gamma=gamma)
    return cellpose(norm_frame, tile=False, normalize=False, diameter=75.0)

## Metrics

Helper function to evaluate tracking score using [Cell Tracking Challenge](celltrackingchallenge.net)'s metrics and annotations.

In [7]:
gt_path = path.parent / f"{dataset}_GT"
gt_data = load_ctc_data(gt_path / "TRA")

def score(output_path: Path) -> Dict:
    return run_metrics(
        gt_data=gt_data, 
        pred_data=load_ctc_data(output_path),
        matcher=CTCMatched,
        metrics=[CTCMetrics],
    )["CTCMetrics"]

Loading TIFFs: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 214.69it/s]


## Parameter Search

Here, we evaluate the segmentation and tracking given multiple values of `gamma`, used on the normalization step before the Cellpose prediction.

In [ ]:
all_labels = []
metrics = []
gammas = [0.1, 0.25, 0.5, 1]
sigma = 5.0

for gamma in gammas:

    # cellpose prediction
    cellpose_labels = np.zeros_like(image, dtype=np.int32)
    array_apply(
        image,
        out_array=cellpose_labels,
        func=predict,
        gamma=gamma,
    )
    all_labels.append(cellpose_labels)
    
    name = f"{dataset}_labels_{gamma}"
    if not COLAB:
        viewer.add_labels(cellpose_labels, name=name, visible=False)

    # cell tracking using `labels` parameter, it's the same as using `labels_to_edges`.
    track(
        config,
        labels=cellpose_labels,
        sigma=sigma,
        overwrite=True
    )

    # exporting to CTC format
    output_path = Path(name.upper()) / "TRA"
    to_ctc(output_path, config, overwrite=True)

    # computing tracking score
    metric = score(output_path)
    metric["gamma"] = gamma
    metrics.append(metric)

print(metrics)

Linking nodes.: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:04<00:00,  6.48it/s]


Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-17
Using GRB solver
Solving ILP batch 0
Constructing ILP ...
Set parameter TimeLimit to value 36000
Solving ILP ...
Set parameter NodeLimit to value 1073741824
Set parameter SolutionLimit to value 1073741824
Set parameter IntFeasTol to value 1e-06
Set parameter Method to value 3
Set parameter MIPGap to value 0.001
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 8824 rows, 14838 columns and 33513 nonzeros
Model fingerprint: 0x8d757e5d
Variable types: 0 continuous, 14838 integer (14838 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-19, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 5366 rows and 7735 columns
Presolve time: 0.08s
Pr

Applying predict ...:  53%|████████████████████████████████████████████████████▎                                             | 16/30 [01:45<01:34,  6.75s/it]

## Combined Contours and Detection

The `labels_to_edges` combines multiple segmentation labels into a single detection and contour map.

The detection map is the maximum value between the binary masks of each label.

The contour map is the average contour map of the binary contours of each label.

In [ ]:
detection, contours = labels_to_edges(all_labels, sigma=sigma)

In [ ]:
if COLAB:
    display(stackview.curtain(image, detection))
else:
    viewer.add_labels(detection)

In [ ]:
if COLAB:
    display(stackview.curtain(image, contours))
else:
    viewer.add_image(contours)

## Tracking

Run the tracking algorithm based on the provided configuration, detected regions, and contours.

In [ ]:
track(
   config,
   detection=detection,
   edges=contours,
   overwrite=True
)

Compute metrics for the multiple hypotheses tracking and compare the scores of the different approaches.

In [ ]:
output_path = Path(f"{dataset}_COMBINED") / "TRA"
to_ctc(output_path, config, overwrite=True)

metric = score(output_path)
metrics.append(metric)

df = pd.DataFrame(metrics)
df.to_csv(f"{dataset}_scores.csv", index=False)
df

## Exporting and Visualization

The intermediate tracking data is stored on disk and must be exported to your preferred format.
Here we convert the resulting tracks to a DataFrame and Zarr to visualize using napari if running locally.

In [ ]:
tracks_df, graph = to_tracks_layer(config)
tracks_df.to_csv(f"{dataset}_tracks.csv", index=False)

segments = tracks_to_zarr(
    config,
    tracks_df,
    overwrite=True,
)

if COLAB:
    display(stackview.curtain(image, segments))
else:
    viewer.add_tracks(
        tracks_df[["track_id", "t", "y", "x"]],
        name="tracks",
        graph=graph,
        visible=True,
    )

    viewer.add_labels(segments, name="segments").contour = 2